In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://mirrors.sonic.net/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xzf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark


import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"


import findspark
findspark.init() #instanciar spark session. Es conveniente tener una unica sesion iniciada y actualizada (si existe la utiliza y si no la crea).
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate() #usamos todos los procesadores locales: local[*]

In [ ]:
spark

In [ ]:
1/2 #comprobar si tengo python 3 (muy recomendable) python3 = 0.5, python2: 0

0.5

In [ ]:
df = spark.read.json("./sample_data/anscombe.json") #output = dataframe cuyo tamaño sera el del json
df.describe() #devuelve un dataframe
df.describe().show() #muestra el contenido
df.describe().toPandas() #tiene tamaño maximo, tener cuidado con archivos reales (grandes). Pandas formatea el output


+-------+------+------------------+------------------+---------------+
|summary|Series|                 X|                 Y|_corrupt_record|
+-------+------+------------------+------------------+---------------+
|  count|    44|                44|                44|              2|
|   mean|  null|               9.0| 7.500454545454546|           null|
| stddev|  null|3.1988369979626783|1.9592439863084525|           null|
|    min|     I|               4.0|               3.1|              [|
|    max|    IV|              19.0|             12.74|              ]|
+-------+------+------------------+------------------+---------------+



,summary,Series,X,Y,_corrupt_record
0,count,44,44,44,2
1,mean,None,9.0,7.500454545454546,None
2,stddev,None,3.1988369979626783,1.9592439863084525,None
3,min,I,4.0,3.1,[
4,max,IV,19.0,12.74,]


In [ ]:
df.show(3) #muestra los primeros 3 elementos
df.printSchema() #muestra el esquema del archivo (informacion principal)

+------+----+----+---------------+
|Series|   X|   Y|_corrupt_record|
+------+----+----+---------------+
|  null|null|null|              [|
|     I|10.0|8.04|           null|
|     I| 8.0|6.95|           null|
+------+----+----+---------------+
only showing top 3 rows

root
 |-- Series: string (nullable = true)
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)
 |-- _corrupt_record: string (nullable = true)



In [ ]:
df2= spark.read.parquet("fichero.parket") #lo bueno de los .parquet es que no tiene tamaño maximo (es un directorio) reparte toda la informacion entre ficheros (como Spark en ordenadores)

Los dataframes no se modifican, es la principal diferencia con Pandas. Esto es lo que permite distrtibuirlo entre ordenadores. Es dificil sincronizar cambios en distintos ordenadores.
Para modificar dataframes crearemos nuevos dataframes basados en otro dataframe.

In [ ]:
df.groupBy("X") #no devuelve un dataframe si no objetos agrupados
df.groupBy("X").agg({"Y":"min", "X":"mean"}).show() #esto ya no es un dataframe sino una tabla con informacion

+----+------+------+
|   X|avg(X)|min(Y)|
+----+------+------+
| 8.0|   8.0|  5.25|
| 7.0|   7.0|  4.81|
|null|  null|  null|
| 4.0|   4.0|   3.1|
|11.0|  11.0|  7.81|
|14.0|  14.0|   8.1|
|19.0|  19.0|  12.5|
|10.0|  10.0|  7.46|
|13.0|  13.0|  7.58|
| 6.0|   6.0|  6.08|
| 5.0|   5.0|  4.74|
| 9.0|   9.0|  7.11|
|12.0|  12.0|  8.15|
+----+------+------+



In [ ]:
#Lo mas parecido a SQL siempre sera lo mas eficiente
df.select("X") #devuelve un dataframe donde solo esta X
df.select("X").count()
df.select(["X", "Y"]).show()

+----+-----+
|   X|    Y|
+----+-----+
|null| null|
|10.0| 8.04|
| 8.0| 6.95|
|13.0| 7.58|
| 9.0| 8.81|
|11.0| 8.33|
|14.0| 9.96|
| 6.0| 7.24|
| 4.0| 4.26|
|12.0|10.84|
| 7.0| 4.81|
| 5.0| 5.68|
|10.0| 9.14|
| 8.0| 8.14|
|13.0| 8.74|
| 9.0| 8.77|
|11.0| 9.26|
|14.0|  8.1|
| 6.0| 6.13|
| 4.0|  3.1|
+----+-----+
only showing top 20 rows



In [ ]:
type (df.select(["X", "Y"]))
type (df["X"]) #columna

pyspark.sql.column.Column

In [ ]:
df.head(2) #devuelve una lista de filas de tamaño (parametro)

[Row(Series=None, X=None, Y=None, _corrupt_record='['),
 Row(Series='I', X=10.0, Y=8.04, _corrupt_record=None)]

In [ ]:
#Crear una columna de tipo dataframe
df.withColumn("z", df["X"]-df["Y"]).show() #recordamos que no se puede modificacr un dataframe, sino que creamos uno nuevo en base a otro

+------+----+-----+---------------+--------------------+
|Series|   X|    Y|_corrupt_record|                   z|
+------+----+-----+---------------+--------------------+
|  null|null| null|              [|                null|
|     I|10.0| 8.04|           null|  1.9600000000000009|
|     I| 8.0| 6.95|           null|  1.0499999999999998|
|     I|13.0| 7.58|           null|                5.42|
|     I| 9.0| 8.81|           null|  0.1899999999999995|
|     I|11.0| 8.33|           null|                2.67|
|     I|14.0| 9.96|           null|   4.039999999999999|
|     I| 6.0| 7.24|           null| -1.2400000000000002|
|     I| 4.0| 4.26|           null| -0.2599999999999998|
|     I|12.0|10.84|           null|  1.1600000000000001|
|     I| 7.0| 4.81|           null|  2.1900000000000004|
|     I| 5.0| 5.68|           null| -0.6799999999999997|
|    II|10.0| 9.14|           null|  0.8599999999999994|
|    II| 8.0| 8.14|           null|-0.14000000000000057|
|    II|13.0| 8.74|           n

In [ ]:
#Renombrar columnas, sobretodo para agregaciones min(x) o avg(y)
df.groupBy("X").agg({"Y":"min", "X":"mean"}).withColumnRenamed("avg(X)", "Final x").show()

+----+-------+------+
|   X|Final x|min(Y)|
+----+-------+------+
| 8.0|    8.0|  5.25|
| 7.0|    7.0|  4.81|
|null|   null|  null|
| 4.0|    4.0|   3.1|
|11.0|   11.0|  7.81|
|14.0|   14.0|   8.1|
|19.0|   19.0|  12.5|
|10.0|   10.0|  7.46|
|13.0|   13.0|  7.58|
| 6.0|    6.0|  6.08|
| 5.0|    5.0|  4.74|
| 9.0|    9.0|  7.11|
|12.0|   12.0|  8.15|
+----+-------+------+



In [ ]:
#Para usar con SQL
df.registerTempTable("Data") #Creamos la tabla para utilizarla como SQL
spark.sql("select Y from Data where X = 10.0").show() #es mejor hacer las querys con SQL

+----+
|   Y|
+----+
|8.04|
|9.14|
|7.46|
+----+



In [ ]:
#consulta con llamada
df.filter(df["X"]>10).show()
#consulta con SQL
spark.sql("select * from Data where X > 10.0").show() #es mejor hacer las querys con SQL
#Ambos son igual de eficientes

+------+----+-----+---------------+
|Series|   X|    Y|_corrupt_record|
+------+----+-----+---------------+
|     I|13.0| 7.58|           null|
|     I|11.0| 8.33|           null|
|     I|14.0| 9.96|           null|
|     I|12.0|10.84|           null|
|    II|13.0| 8.74|           null|
|    II|11.0| 9.26|           null|
|    II|14.0|  8.1|           null|
|    II|12.0| 9.13|           null|
|   III|13.0|12.74|           null|
|   III|11.0| 7.81|           null|
|   III|14.0| 8.84|           null|
|   III|12.0| 8.15|           null|
|    IV|19.0| 12.5|           null|
+------+----+-----+---------------+

+------+----+-----+---------------+
|Series|   X|    Y|_corrupt_record|
+------+----+-----+---------------+
|     I|13.0| 7.58|           null|
|     I|11.0| 8.33|           null|
|     I|14.0| 9.96|           null|
|     I|12.0|10.84|           null|
|    II|13.0| 8.74|           null|
|    II|11.0| 9.26|           null|
|    II|14.0|  8.1|           null|
|    II|12.0| 9.13|        

In [66]:
#Ejercicio Wallmart
data = spark.read.csv("./walmart_stock.csv").show() #carga del csv


+----------+------------------+------------------+------------------+------------------+--------+------------------+
|       _c0|               _c1|               _c2|               _c3|               _c4|     _c5|               _c6|
+----------+------------------+------------------+------------------+------------------+--------+------------------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|         59.549999|         58.91

In [69]:
data = spark.read.options(header=True).csv("./walmart_stock.csv").show() #carga del csv con cabeceras formateadas

+----------+------------------+------------------+------------------+------------------+--------+------------------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000000004|
|2012-01-10|             59.43|59.709998999999996|             5

In [72]:
data = spark.read.options(header=True).csv("./walmart_stock.csv").printSchema() 
data = spark.read.options(header=True, inferSchema=True).csv("./walmart_stock.csv").printSchema() #cuando imprimimos el esquema sale que es todo string, con la opcion inferSchema=True salen los tipos de verdad


root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



Hacer ejercicio campus. Ojo con el 7, hay que mirar la documentacion!!